<a href="https://colab.research.google.com/github/mauricio-ms/motor-imagery-convolutional-recurrent-neural-network/blob/master/notebooks/cnn-1d-tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Verify if TPU is enabled

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.3.0


Mount Google Drive directory

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Unzip dataset

In [3]:
!tar -xf "drive/My Drive/motor-imagery-convolutional-recurrent-neural-network/raw-csv-files.tar.xz"

^C


Install needed packages

In [2]:
!pip install colorlog

In [2]:
import logging
import colorlog


def get_logger(name="__main__", debug=False):
    log_format = (
        "[%(levelname)s] "
        "%(asctime)s - "
        "%(name)s: "
        "%(funcName)s - "
        "%(message)s"
    )
    bold_seq = "\033[1m"
    colorlog_format = (
        f"{bold_seq} "
        "%(log_color)s "
        f"{log_format}"
    )
    colorlog.basicConfig(format=colorlog_format)

    logger = logging.getLogger(name)
    if debug:
        logger.setLevel(logging.DEBUG)
    else:
        logger.setLevel(logging.INFO)

    # Output full log
    fh = logging.FileHandler("app.log")
    fh.setLevel(logging.DEBUG)
    formatter = logging.Formatter(log_format)
    fh.setFormatter(formatter)
    logger.addHandler(fh)

    return logger

In [3]:
import os
import numpy as np

LOGGER = get_logger("physionet.py")
N_CHANNELS = 64
PREPROCESSED_CSV_FILES_DIR = "raw-csv-files"

In [4]:
def load_data(train_size=0.75, validation_size=None, n_subjects=None, **kwargs):
    LOGGER.info("Loading Physionet dataset ...")
    subjects = np.array(sorted(os.listdir(PREPROCESSED_CSV_FILES_DIR)))
    if n_subjects is not None:
        subjects = subjects[:n_subjects]
    train_subjects, test_subjects = _train_test_split_subjects(subjects, train_size)
    if validation_size is not None:
        train_subjects, validation_subjects = _train_test_split_subjects(train_subjects, 1-validation_size)
        LOGGER.info(f"(Train, Validation, Test) Subjects = "
                    f"({len(train_subjects)}, {len(validation_subjects)}, {len(test_subjects)})")
        return _load_set(train_subjects, **kwargs), \
            _load_set(validation_subjects, **kwargs), \
            _load_set(test_subjects, **kwargs)

    LOGGER.info(f"(Train, Validation) Subjects = ({len(train_subjects)}, {len(test_subjects)})")
    return _load_set(train_subjects, **kwargs), _load_set(test_subjects, **kwargs)


def _train_test_split_subjects(subjects, train_size):
    train_subjects_mask = np.random.rand(len(subjects)) < train_size
    return subjects[train_subjects_mask], subjects[~train_subjects_mask]


def _load_set(subjects, n_readers=5, n_parse_threads=5, batch_size=100,
              convert_to_2d=False, expand_dim=False):
    path_files = [os.path.join(PREPROCESSED_CSV_FILES_DIR, subject, file_name)
                  for subject in subjects
                  for file_name in sorted(os.listdir(os.path.join(PREPROCESSED_CSV_FILES_DIR, subject)))]
    dataset = tf.data.Dataset.list_files(path_files)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers)
    dataset = dataset.map(lambda r: _preprocess(r, convert_to_2d=convert_to_2d,
                                                expand_dim=expand_dim),
                          num_parallel_calls=n_parse_threads)
    return dataset.batch(batch_size).prefetch(1)


def _preprocess(eeg_record, convert_to_2d=False, expand_dim=False):
    # Create the definitions for the columns (channels + label)
    # The empty array tells TensorFlow to raise exception to missing values
    defs = [tf.constant([], dtype=tf.float32)] * (N_CHANNELS + 1)
    fields = tf.io.decode_csv(eeg_record, record_defaults=defs)
    x = _get_features(fields, convert_to_2d=convert_to_2d)
    if expand_dim:
        x = x[..., np.newaxis]
    y = tf.cast(tf.stack(fields[-1:]), tf.int32)
    return x, y


def _get_features(fields, convert_to_2d=False):
    return tf.stack(fields[:-1])

In [5]:
from tensorflow import keras

batch_size=500

train_set, validation_set, test_set = load_data(expand_dim=True, 
                                                validation_size=0.20,
                                                batch_size=batch_size)

  [INFO] 2020-08-13 23:15:50,517 - physionet.py: load_data - Loading Physionet dataset ...
  [INFO] 2020-08-13 23:15:50,519 - physionet.py: load_data - (Train, Validation, Test) Subjects = (66, 16, 26)


In [6]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

strategy = tf.distribute.experimental.TPUStrategy(tpu)
print("REPLICAS: ", strategy.num_replicas_in_sync)

with strategy.scope():
  model = keras.models.Sequential([
      keras.layers.Conv1D(32, 3, activation="relu",
                          kernel_initializer="he_normal",
                          padding="SAME",
                          input_shape=[64, 1]),
      keras.layers.Conv1D(64, 3, activation="relu",
                          kernel_initializer="he_normal",
                          padding="SAME"),
      keras.layers.Conv1D(128, 3, activation="relu",
                          kernel_initializer="he_normal",
                          padding="SAME"),
      keras.layers.Flatten(),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(1024, activation="relu",
                        kernel_initializer="he_normal"),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(5, activation="softmax")
  ])

  optimizer = keras.optimizers.Adam(lr=1e-4)
  model.compile(loss="sparse_categorical_crossentropy",
                optimizer=optimizer,
                metrics=["accuracy"])

Running on TPU  ['10.26.123.218:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.26.123.218:8470


  [INFO] 2020-08-13 23:15:55,868 - tensorflow: initialize_tpu_system - Initializing the TPU system: grpc://10.26.123.218:8470


INFO:tensorflow:Clearing out eager caches


  [INFO] 2020-08-13 23:16:11,964 - tensorflow: initialize_tpu_system - Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


  [INFO] 2020-08-13 23:16:12,020 - tensorflow: _query_tpu_system_metadata - *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


  [INFO] 2020-08-13 23:16:12,021 - tensorflow: _query_tpu_system_metadata - *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [7]:
LOGGER.info("Starting training CNN 1D model ...")
model.fit(train_set, epochs=2,
          validation_data=validation_set)
LOGGER.info("Training CNN 1D model end!")

  [INFO] 2020-08-13 23:16:13,493 - physionet.py: <module> - Starting training CNN 1D model ...


InvalidArgumentError: ignored

In [8]:
train_set

<PrefetchDataset shapes: ((None, 5, 1), (None, 1)), types: (tf.float32, tf.int32)>